#**MNIST- refers to image identification of hand written digits data**

**This project focuses on creating a neural netwrok model completly using numpy instead pof tensorflow library**

As for the data set we can either get it through kaggle, keras, or even openml
In this project we are using openML since we do not want to use keras in this complete code

##Some conventions
Upper case letters refres to 2D matrix datas. Ex- shape( samples, features )
Lower case refrs to 1D vectors. Ex- shape(label)

##Forward Propagation
Each layer has an forward prop where it sends data to the next layer after getting output from the uploaded input, goes from first layer to end
#Backward Propagation
works from last layer to first where the derivatives are sent backward to update the weights and bias and sent again until loss is decreased


In [1]:

import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

#Load and seperate data for train and test

The data will be a 28x28 pixel image's greyscale intensity data (0-255 for each pixel) for each number which is filled with  black to white colors, which we refer as 0-1. Where 0 refers to black and 1 refers to white, but in system recoginition the value is given as its density value which is 0 for black and 255 for white, hence to use the data set we have to do these steps

First, the openml data is given as 
X is a float of 784 (28x28) values per row that is 784 features for each sample, and y is a list of strings - ('0','1',2'....,'9')
hence we convert y label values to int from string
then divide then x data by 255 to get values between 0 and 1.
(x value is color that is 01 and y value is 0-9 number labels)

In [2]:
#Load MNIST data from openML
X, y = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False)
#Convert string to int
y = y.astype(np.int64)
#Normalize pixel value that is 0 to 255 is changed to 0-1
X = X / 255.0
#split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=10000, random_state=42, stratify=y)
#In above, random_state=42 refers to random splitting for better working, the 42 can be any number
# stratify = y is used to arrange y values such that it is not too biased in either test or train set ( that is the same label (0-9) is spread evenly in percentage)

print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")
#this data set has 60k sample each of 784px hence those many features for each, test data set is 10k as given above 

Train shape: (60000, 784), Test shape: (10000, 784)


#Batch Generator
###Neural network do not train on the entire dataset at once - too slow and too much memory
INSTEAD WE USE MINI BATCHES, taht is train 32-64 data per batch and then take another batch while removing the previous batch from emory - this helps when the dataset is large.

ex: when we have 60k images we train 32 to 64 images per batch

In [3]:
def batch_generator(X, y, batch_size=32):
    n_samples = X.shape[0] #total number of samples
    indices = np.arange(n_samples)   # this creates a list with the specified number of terms like [0,1,2,3,...]
    np.random.shuffle(indices)       # shuffle the list for randomness
    
    for start in range(0, n_samples, batch_size):
        end = start + batch_size
        batch_ids = indices[start:end]
        yield X[batch_ids], y[batch_ids] #yield is used such that memory is saved since it outputs one step then forgets memory and does it for the next batch unlike how return does is send it as a largee single block
#for those with doubts how X[[23,45,67,32,43,..]] this type of data gives output - this works as X[23],X[45],... note that this is how np.array works and will noyt work with python lists

#Layer Dense

For this we will be using class instead of direct functions

##For those who do not have any basic idea of class-
in simple words it is where we group multiple functions all having connected variable for each object (child of the class) created/initialized
the first function called __init__ runs eventhough it is not called whenever an object is defined, meaning it initialises the data such as the basic variables that are used by that object's functions
the functions inside the object are accessed by '.' operator

##Why use a class for dense
A Dense (fully connected) layer needs:
- A set of **weights** and **biases** for each layer.
- A **forward method** to compute outputs.
- A **backward method** to compute gradients (for training).


In [ ]:
#object = DenseLayer(n_iputs, n_neurons) these parameter values are the __init__ function's parameter
class DenseLayer:
    def __init__(self, n_inputs, n_neurons):
        """
        A fully connected (Dense) layer.
        n_inputs: number of features (e.g. 784 for MNIST)
        n_neurons: number of neurons/units in this layer
        """
        # Initialize weights small random values
        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)#creates a matrix of random numbers as value which is multipled by 0.01 to reduce its size, the matrix shape is (n_iputs,n_neurons)
        # matrix shape here means that each of the n_neurons have n_inputs number of weights
        # Initialize biases as zeros
        self.biases = np.zeros((1, n_neurons)) #creates a array of zeroes for the guven shape (1,number of units in the layer)

    def forward(self, inputs):
        """
        Forward pass through the layer.
        inputs: shape (batch_size, n_inputs)
        """
        self.inputs = inputs  # keep for backward prop, input refers to the acivataion 'a vector' passed to this layer by the previous layer 
        self.output = np.dot(inputs, self.weights) + self.biases

    def backward(self, dvalues):#d hear means symbol for differentiation
        # dvalues: gradient from the next layer (∂L/∂output), 
        # similar to "y_hat - y" (that only shows up at the loss layer).
        self.dweights = np.dot(self.inputs.T, dvalues)   # ∂L/∂W, .T means transpose, where the value of diff wrt w is summation of (y hat - y )* x_i
        # Each weight’s gradient is how much changing that weight changes the loss
        self.dbiases = np.sum(dvalues, axis=0, keepdims=True)  # ∂L/∂b, the value of diff wrt b is summation of (y cap - y )
        # Gradient w.r.t. inputs: dL/dout · Wᵀ
        # This passes the gradient backward so earlier layers can update too    
        self.dinputs = np.dot(dvalues, self.weights.T)  
        #and the prevoius layer uses these dvalues and updates its weight and bias 


#Layer - ReLu Activation

ReLu function g(x)=max(0,x)
We store the input for later add ons such as backward propagation
##Backward prop
∂𝐿/∂𝑦 = gradient coming from the next layer, that’s what you called dvalues or doutput.
∂𝑦/∂𝑥 = derivative of the activation function (e.g., ReLU).
∂𝐿/∂𝑥 = gradient wrt the input (what we need to send further back) that’s dinputs, which is a product of the first two

In [ ]:
class ActivationReLU:
    def forward(self, inputs):
        # Save inputs for backprop
        self.inputs = inputs
        # Relu function is max(0,x)
        self.output = np.maximum(0, inputs)
        
    def backward(self, dvalues):
        self.dinputs = dvalues.copy()#copy values of the next layer's gradient/derivative
        self.dinputs[self.inputs <= 0] = 0
        '''
        inputs  = [-2, -1,  0,  3,  5] output for this [0,0,0,3,5] as per Relu
        dvalues = [ 1,  2,  3,  4,  5] this is the derivative or gradient passed from the next layer
        dinputs = [ 0,  0,  0,  4,  5] wherever the input is negative the dinput is also zero, and whenevr it is positive we get the dvalue
        relu = f(x)= 0 if x is <=0, f(x)=x, x>0
        derivative of f(x) = 0 if x is <=0, derivative of f(x)=1, x>0
        '''

#Layer - Softmax Activation

Softmax - is where the input gives a output between 0 to 1 (probabilities that sum up to 1)with the formula e^zi/sum of e^zis, here the input is the 'activation 'a' matrix of vectors't we get as output from the previous layer

In [6]:
class ActivationSoftmax:
    def forward(self, inputs):
        # Save inputs for backprop
        self.inputs = inputs
        
        # inputs is a matrix of multiple samples with multiple features in a row, here np.exp also ceate the matrix of same dimension but with gtheir exponent value as the output
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        #above we subtract each value in each row of input by the max value in the input's each row so that if the values are too learge like 1000 then e^1000 will be too big so we normalize it 
        
        probabilities = exp_values / np.sum(exp_values, axis=1, keepdims=True)
        #axis=1, refers to row wise sums that divide each value in that row in the matrix
        
        self.output = probabilities
    
    def backward(self, dvalues):
        # Rarely used directly since we combine with loss
        self.dinputs = dvalues.copy()

#Layer Loss CategoricalCrossEntropy

Used to calculate loss of Softmax activation, calculated by 1/N * sum of (log(prediction_i))) that is mean formula
for a digit (like '2') the y_true can be [0,0,1,0,0,0,0,0,0,0] called one hot encoding or [0,1,2,3,4,5,6,7,8,9] 

##Example with 3 digits input [0,1,2]
y_pred = np.array([
    [0.1, 0.7, 0.2],   ### model says: "likely class 1"
    [0.8, 0.1, 0.1],   ### model says: "likely class 0"
    [0.2, 0.2, 0.6]    ### model says: "likely class 2"
])
### shape = (3 samples, 3 classes)
True labels (y_true):
Case A → as integers:

python
Copy code
y_true = np.array([1, 0, 2])   ### correct classes are 1, 0, 2
### shape = (3,)
Case B → one-hot encoded:

python
Copy code
y_true = np.array([
    [0, 1, 0],   ### class 1
    [1, 0, 0],   ### class 0
    [0, 0, 1]    ### class 2
])
### shape = (3,3)

In [ ]:
class Loss_CategoricalCrossentropy:
    def forward(self, y_pred, y_true):
        # clip values to prevent log(0), that is the value is in range 1^-7 to 1 - 1^-7
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)

        # if labels are one-hot encoded [0,0,1,0,0,0,0,0,0,0],[....],[...] means class '2' in first row, and like this for each sample, like this for each sample
        if len(y_true.shape) == 2:#since it is 2D matrix with shape like (10,10)- 10 rows and 10 column , and the length of (10,10) is 2
            correct_confidences = np.sum(y_pred_clipped * y_true, axis=1)
        # if labels are just class indices [1,0,2...] where number of terms is same as number of samples, and the number is the integer that has the most probability for each sample
        elif len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[range(len(y_pred)), y_true] #range(3)= [0,1,2], here y_pr....[..,..]=[[0,1,2],[1,0,2]]=[[0,1],[1,0],[2,2]]

        # loss = negative log likelihood
        neg_log_likelihoods = -np.log(correct_confidences)
        return np.mean(neg_log_likelihoods) #1/N sum of logs
    
    def backward(self, dvalues, y_true):
        samples = len(dvalues)       # number of samples in the batch
        labels = len(dvalues[0])     # number of classes (labels)

        if len(y_true.shape) == 1:
            y_true = np.eye(labels)[y_true]
            #np.eye creates diagonal matrix [1,0,0],[0,1,0],[0,0,1]
            #[y_true] whose value are the labels that are most propable like (0,2) chooses that row of the diagonal matrix to change into one hot encoding 
            #np.eye(3)[y_true] makes [1,2] meaning the first propability hiogh for number label 1 and next is for 2 is converted to one hot encoding ([[0,1,0],[0,0,1]])
        self.dinputs = -y_true / dvalues#∂L/∂y_pred = -y_true/y_pred
        self.dinputs = self.dinputs / samples


#Optimizer_Adam (complex mathematical formulas for beginner level, better to use the library rather than confuse yourselves)
As for definition an optimizer helps reduce the number of steops to reach minima by adjusting the learning rate so that it does not diverge but meets the minima at a faster rate

In [ ]:
class Optimizer_Adam:
    def __init__(self, learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-8):
        self.learning_rate = learning_rate
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.iterations = 0   # to keep track of steps

    def update_params(self, layer):
        # If first time, initialize momentums and caches
        if not hasattr(layer, 'm_w'):
            layer.m_w = np.zeros_like(layer.weights)
            layer.v_w = np.zeros_like(layer.weights)
            layer.m_b = np.zeros_like(layer.biases)
            layer.v_b = np.zeros_like(layer.biases)

        # Update step count
        self.iterations += 1

        # Momentum update for weights and biases
        layer.m_w = self.beta1 * layer.m_w + (1 - self.beta1) * layer.dweights
        layer.m_b = self.beta1 * layer.m_b + (1 - self.beta1) * layer.dbiases

        # RMSProp cache update
        layer.v_w = self.beta2 * layer.v_w + (1 - self.beta2) * (layer.dweights ** 2)
        layer.v_b = self.beta2 * layer.v_b + (1 - self.beta2) * (layer.dbiases ** 2)

        # Correct bias for small t
        m_w_corr = layer.m_w / (1 - self.beta1 ** self.iterations)
        m_b_corr = layer.m_b / (1 - self.beta1 ** self.iterations)
        v_w_corr = layer.v_w / (1 - self.beta2 ** self.iterations)
        v_b_corr = layer.v_b / (1 - self.beta2 ** self.iterations)

        # Update parameters
        layer.weights -= self.learning_rate * m_w_corr / (np.sqrt(v_w_corr) + self.epsilon)
        layer.biases  -= self.learning_rate * m_b_corr / (np.sqrt(v_b_corr) + self.epsilon)
